In [1]:
import numpy as np
import pandas as pd

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import LabelEncoder

from imblearn import under_sampling, over_sampling
from imblearn.over_sampling import SMOTENC

In [2]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

2024-06-10 17:56:17.251486: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-10 17:56:20.710911: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-06-10 17:56:20.710943: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-06-10 17:56:27.168548: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directo

In [3]:
df = pd.read_csv("./dataset_esi_with_chief_complaint.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 560486 entries, 0 to 560485
Columns: 212 entries, age to cc_wristpain
dtypes: float64(209), object(3)
memory usage: 906.5+ MB


In [4]:
df.sample(10)

,age,gender,dep_name,arrivalmode,triage_vital_hr,triage_vital_sbp,triage_vital_dbp,triage_vital_rr,triage_vital_o2,triage_vital_o2_device,...,cc_vaginaldischarge,cc_vaginalpain,cc_weakness,cc_wheezing,cc_withdrawal-alcohol,cc_woundcheck,cc_woundinfection,cc_woundre-evaluation,cc_wristinjury,cc_wristpain
355640,52.0,Female,A,Car,70.0,131.0,75.0,16.0,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
155484,21.0,Male,A,ambulance,96.0,136.0,77.0,16.0,98.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
164043,32.0,Male,C,Car,99.0,138.0,88.0,NaN,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98956,34.0,Female,A,ambulance,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
558829,90.0,Female,A,ambulance,56.0,169.0,76.0,14.0,99.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
288896,18.0,Male,A,ambulance,100.0,116.0,78.0,20.0,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
239073,88.0,Female,B,ambulance,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
120140,22.0,Female,A,Car,78.0,131.0,74.0,18.0,99.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
112747,31.0,Female,A,Walk-in,78.0,118.0,82.0,18.0,99.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
143481,52.0,Male,B,Walk-in,104.0,123.0,71.0,20.0,97.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
# Are there any null values?
df.isnull().values.any()

True

In [6]:
def missing_values_table(dataframe):
    variables_with_na = [col for col in dataframe.columns if dataframe[col].isnull().sum() > 0]
    n_miss = dataframe[variables_with_na].isnull().sum().sort_values(ascending=False)
    ratio = (dataframe[variables_with_na].isnull().sum() / dataframe.shape[0] * 100).sort_values(ascending=False)
    missing_df = pd.concat([n_miss, np.round(ratio, 2)], axis=1, keys=['n_miss', 'ratio'])
    print(missing_df)
    return variables_with_na

missing_values_table(df)

                        n_miss  ratio
triage_vital_o2         271317  48.41
triage_vital_o2_device  226855  40.47
triage_vital_temp       182985  32.65
triage_vital_rr         170487  30.42
triage_vital_dbp        168063  29.99
...                        ...    ...
cc_medicalproblem         2585   0.46
cc_mass                   2585   0.46
cc_maleguproblem          2585   0.46
esi                       2457   0.44
age                         11   0.00

[210 rows x 2 columns]


['age',
 'arrivalmode',
 'triage_vital_hr',
 'triage_vital_sbp',
 'triage_vital_dbp',
 'triage_vital_rr',
 'triage_vital_o2',
 'triage_vital_o2_device',
 'triage_vital_temp',
 'esi',
 'cc_abdominalcramping',
 'cc_abdominaldistention',
 'cc_abdominalpain',
 'cc_abdominalpainpregnant',
 'cc_abnormallab',
 'cc_abscess',
 'cc_addictionproblem',
 'cc_agitation',
 'cc_alcoholintoxication',
 'cc_alcoholproblem',
 'cc_allergicreaction',
 'cc_alteredmentalstatus',
 'cc_animalbite',
 'cc_ankleinjury',
 'cc_anklepain',
 'cc_anxiety',
 'cc_arminjury',
 'cc_armpain',
 'cc_armswelling',
 'cc_assaultvictim',
 'cc_asthma',
 'cc_backpain',
 'cc_bleeding/bruising',
 'cc_blurredvision',
 'cc_bodyfluidexposure',
 'cc_breastpain',
 'cc_breathingdifficulty',
 'cc_breathingproblem',
 'cc_burn',
 'cc_cardiacarrest',
 'cc_cellulitis',
 'cc_chestpain',
 'cc_chesttightness',
 'cc_chills',
 'cc_coldlikesymptoms',
 'cc_confusion',
 'cc_conjunctivitis',
 'cc_constipation',
 'cc_cough',
 'cc_cyst',
 'cc_decreasedblo

In [7]:
df = df.dropna()

In [8]:
new_df = df.drop(['dep_name', 'arrivalmode'], axis=1)

In [9]:
like_num = [col for col in new_df.columns if new_df[col].dtypes != 'O' and len(new_df[col].value_counts()) > 10]
cols_need_scale = [col for col in new_df.columns if col not in like_num
                   and col not in 'esi']

print('like_num: ', like_num)
print("cols_need_scale: ", cols_need_scale)

like_num:  ['age', 'triage_vital_hr', 'triage_vital_sbp', 'triage_vital_dbp', 'triage_vital_rr', 'triage_vital_o2', 'triage_vital_temp']
cols_need_scale:  ['gender', 'triage_vital_o2_device', 'cc_abdominalcramping', 'cc_abdominaldistention', 'cc_abdominalpain', 'cc_abdominalpainpregnant', 'cc_abnormallab', 'cc_abscess', 'cc_addictionproblem', 'cc_agitation', 'cc_alcoholintoxication', 'cc_alcoholproblem', 'cc_allergicreaction', 'cc_alteredmentalstatus', 'cc_animalbite', 'cc_ankleinjury', 'cc_anklepain', 'cc_anxiety', 'cc_arminjury', 'cc_armpain', 'cc_armswelling', 'cc_assaultvictim', 'cc_asthma', 'cc_backpain', 'cc_bleeding/bruising', 'cc_blurredvision', 'cc_bodyfluidexposure', 'cc_breastpain', 'cc_breathingdifficulty', 'cc_breathingproblem', 'cc_burn', 'cc_cardiacarrest', 'cc_cellulitis', 'cc_chestpain', 'cc_chesttightness', 'cc_chills', 'cc_coldlikesymptoms', 'cc_confusion', 'cc_conjunctivitis', 'cc_constipation', 'cc_cough', 'cc_cyst', 'cc_decreasedbloodsugar-symptomatic', 'cc_dehydr

In [10]:
changeMaxToOne = ['triage_vital_o2_device', 'cc_abdominalcramping', 'cc_abdominaldistention', 'cc_abdominalpain', 'cc_abdominalpainpregnant', 'cc_abnormallab', 'cc_abscess', 'cc_addictionproblem', 'cc_agitation', 'cc_alcoholintoxication', 'cc_alcoholproblem', 'cc_allergicreaction', 'cc_alteredmentalstatus', 'cc_animalbite', 'cc_ankleinjury', 'cc_anklepain', 'cc_anxiety', 'cc_arminjury', 'cc_armpain', 'cc_armswelling', 'cc_assaultvictim', 'cc_asthma', 'cc_backpain', 'cc_bleeding/bruising', 'cc_blurredvision', 'cc_bodyfluidexposure', 'cc_breastpain', 'cc_breathingdifficulty', 'cc_breathingproblem', 'cc_burn', 'cc_cardiacarrest', 'cc_cellulitis', 'cc_chestpain', 'cc_chesttightness', 'cc_chills', 'cc_coldlikesymptoms', 'cc_confusion', 'cc_conjunctivitis', 'cc_constipation', 'cc_cough', 'cc_cyst', 'cc_decreasedbloodsugar-symptomatic', 'cc_dehydration', 'cc_dentalpain', 'cc_depression', 'cc_detoxevaluation', 'cc_diarrhea', 'cc_dizziness', 'cc_drug/alcoholassessment', 'cc_drugproblem', 'cc_dyspnea', 'cc_dysuria', 'cc_earpain', 'cc_earproblem', 'cc_edema', 'cc_elbowpain', 'cc_elevatedbloodsugar-nosymptoms', 'cc_elevatedbloodsugar-symptomatic', 'cc_emesis', 'cc_epigastricpain', 'cc_epistaxis', 'cc_exposuretostd', 'cc_extremitylaceration', 'cc_extremityweakness', 'cc_eyeinjury', 'cc_eyepain', 'cc_eyeproblem', 'cc_eyeredness', 'cc_facialinjury', 'cc_faciallaceration', 'cc_facialpain', 'cc_facialswelling', 'cc_fall', 'cc_fall>65', 'cc_fatigue', 'cc_femaleguproblem', 'cc_fever', 'cc_fever-75yearsorolder', 'cc_fever-9weeksto74years', 'cc_feverimmunocompromised', 'cc_fingerinjury', 'cc_fingerpain', 'cc_fingerswelling', 'cc_flankpain', 'cc_follow-upcellulitis', 'cc_footinjury', 'cc_footpain', 'cc_footswelling', 'cc_foreignbodyineye', 'cc_fulltrauma', 'cc_generalizedbodyaches', 'cc_gibleeding', 'cc_giproblem', 'cc_groinpain', 'cc_hallucinations', 'cc_handinjury', 'cc_handpain', 'cc_headache', 'cc_headache-newonsetornewsymptoms', 'cc_headache-recurrentorknowndxmigraines', 'cc_headachere-evaluation', 'cc_headinjury', 'cc_headlaceration', 'cc_hematuria', 'cc_hemoptysis', 'cc_hippain', 'cc_homicidal', 'cc_hyperglycemia', 'cc_hypertension', 'cc_hypotension', 'cc_influenza', 'cc_ingestion', 'cc_insectbite', 'cc_irregularheartbeat', 'cc_jawpain', 'cc_jointswelling', 'cc_kneeinjury', 'cc_kneepain', 'cc_laceration', 'cc_leginjury', 'cc_legpain', 'cc_legswelling', 'cc_lethargy', 'cc_lossofconsciousness', 'cc_maleguproblem', 'cc_mass', 'cc_medicalproblem', 'cc_medicalscreening', 'cc_medicationproblem', 'cc_medicationrefill', 'cc_migraine', 'cc_modifiedtrauma', 'cc_motorcyclecrash', 'cc_motorvehiclecrash', 'cc_multiplefalls', 'cc_nasalcongestion', 'cc_nausea', 'cc_nearsyncope', 'cc_neckpain', 'cc_neurologicproblem', 'cc_numbness', 'cc_oralswelling', 'cc_otalgia', 'cc_other', 'cc_overdose-accidental', 'cc_overdose-intentional', 'cc_pain', 'cc_palpitations', 'cc_panicattack', 'cc_pelvicpain', 'cc_poisoning', 'cc_post-opproblem', 'cc_psychiatricevaluation', 'cc_psychoticsymptoms', 'cc_rapidheartrate', 'cc_rash', 'cc_rectalbleeding', 'cc_rectalpain', 'cc_respiratorydistress', 'cc_ribinjury', 'cc_ribpain', 'cc_seizure-newonset', 'cc_seizure-priorhxof', 'cc_seizures', 'cc_shortnessofbreath', 'cc_shoulderinjury', 'cc_shoulderpain', 'cc_sicklecellpain', 'cc_sinusproblem', 'cc_skinirritation', 'cc_skinproblem', 'cc_sorethroat', 'cc_stdcheck', 'cc_strokealert', 'cc_suicidal', 'cc_suture/stapleremoval', 'cc_swallowedforeignbody', 'cc_syncope', 'cc_tachycardia', 'cc_testiclepain', 'cc_thumbinjury', 'cc_tickremoval', 'cc_toeinjury', 'cc_toepain', 'cc_trauma', 'cc_unresponsive', 'cc_uri', 'cc_urinaryfrequency', 'cc_urinaryretention', 'cc_urinarytractinfection', 'cc_vaginalbleeding', 'cc_vaginaldischarge', 'cc_vaginalpain', 'cc_weakness', 'cc_wheezing', 'cc_withdrawal-alcohol', 'cc_woundcheck', 'cc_woundinfection', 'cc_woundre-evaluation', 'cc_wristinjury', 'cc_wristpain']

new_df[changeMaxToOne] = new_df[changeMaxToOne].clip(upper=1)

In [11]:
def one_hot_encoder(dataframe, categorical_columns, nan_as_category=False):
    original_columns = list(dataframe.columns)
    dataframe = pd.get_dummies(dataframe, columns=categorical_columns,
                               dummy_na=nan_as_category, drop_first=True , dtype=int)
    new_columns = [col for col in dataframe.columns if col not in original_columns]
    return dataframe, new_columns
    
new_df, one_hot_encodeds = one_hot_encoder(new_df, cols_need_scale)

# Without Robust Scaler

In [12]:
X = new_df.drop('esi', axis=1)
y = np.ravel(new_df[['esi']])

In [13]:
for idx, col in enumerate(X.columns):
    print(f"Index: {idx}, Column Name: {col}")

Index: 0, Column Name: age
Index: 1, Column Name: triage_vital_hr
Index: 2, Column Name: triage_vital_sbp
Index: 3, Column Name: triage_vital_dbp
Index: 4, Column Name: triage_vital_rr
Index: 5, Column Name: triage_vital_o2
Index: 6, Column Name: triage_vital_temp
Index: 7, Column Name: gender_Male
Index: 8, Column Name: triage_vital_o2_device_1.0
Index: 9, Column Name: cc_abdominalcramping_1.0
Index: 10, Column Name: cc_abdominaldistention_1.0
Index: 11, Column Name: cc_abdominalpain_1.0
Index: 12, Column Name: cc_abdominalpainpregnant_1.0
Index: 13, Column Name: cc_abnormallab_1.0
Index: 14, Column Name: cc_abscess_1.0
Index: 15, Column Name: cc_addictionproblem_1.0
Index: 16, Column Name: cc_agitation_1.0
Index: 17, Column Name: cc_alcoholintoxication_1.0
Index: 18, Column Name: cc_alcoholproblem_1.0
Index: 19, Column Name: cc_allergicreaction_1.0
Index: 20, Column Name: cc_alteredmentalstatus_1.0
Index: 21, Column Name: cc_animalbite_1.0
Index: 22, Column Name: cc_ankleinjury_1.0
I

In [14]:
categorical_features = list(range(7, 209))
print(categorical_features)

[7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208]


In [15]:
X.sample(10)

,age,triage_vital_hr,triage_vital_sbp,triage_vital_dbp,triage_vital_rr,triage_vital_o2,triage_vital_temp,gender_Male,triage_vital_o2_device_1.0,cc_abdominalcramping_1.0,...,cc_vaginaldischarge_1.0,cc_vaginalpain_1.0,cc_weakness_1.0,cc_wheezing_1.0,cc_withdrawal-alcohol_1.0,cc_woundcheck_1.0,cc_woundinfection_1.0,cc_woundre-evaluation_1.0,cc_wristinjury_1.0,cc_wristpain_1.0
287483,61.0,92.0,138.0,80.0,18.0,99.0,97.9,1,0,0,...,0,0,0,0,0,1,0,0,0,0
325387,61.0,64.0,178.0,95.0,17.0,98.0,97.5,0,0,0,...,0,0,0,0,0,0,0,0,0,0
270999,24.0,62.0,109.0,55.0,18.0,99.0,98.1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
232121,46.0,77.0,126.0,88.0,16.0,97.0,97.3,1,0,0,...,0,0,0,0,0,0,0,0,0,0
312057,67.0,89.0,157.0,93.0,17.0,97.5,97.0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
193799,57.0,60.0,164.0,113.0,20.0,98.0,98.2,1,0,0,...,0,0,0,0,0,0,0,0,0,0
225989,77.0,76.0,126.5,70.5,18.0,95.5,98.1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
506905,49.0,98.0,167.0,108.0,16.0,98.0,98.1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
490170,22.0,92.0,115.0,76.0,18.0,99.0,98.8,0,0,0,...,0,0,0,0,0,0,0,0,0,0
52618,44.0,112.0,97.0,50.0,22.0,94.0,98.7,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
sm = SMOTENC(random_state=42, categorical_features=categorical_features)
X, y = sm.fit_resample(X, y)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state = 357)

print("Done bang")

In [ ]:
print("real bang")

In [ ]:
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_train = to_categorical(y_train)

y_test = label_encoder.transform(y_test)
y_test = to_categorical(y_test)

In [ ]:
y_train_df = pd.DataFrame(y_train, columns=['class_0', 'class_1', 'class_2', 'class_3', 'class_4'])
train_df = pd.concat([X_train.reset_index(drop=True), y_train_df.reset_index(drop=True)], axis=1)
train_df.to_csv('train_data_ESI_without_robust_scaler.csv', index=False)

In [ ]:
y_test_df = pd.DataFrame(y_test, columns=['class_0', 'class_1', 'class_2', 'class_3', 'class_4'])
test_df = pd.concat([X_test.reset_index(drop=True), y_test_df.reset_index(drop=True)], axis=1)
test_df.to_csv('test_data_ESI_without_robust_scaler.csv', index=False)

# With RobustScaler

In [ ]:
X = new_df.drop('esi', axis=1)
y = np.ravel(new_df[['esi']])

In [ ]:
robustScaler = RobustScaler()
X[like_num] = robustScaler.fit_transform(X[like_num])

In [ ]:
sm = SMOTENC(random_state=42, categorical_features=categorical_features)
X, y = sm.fit_resample(X, y)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state = 357)

print("Done bang")

In [ ]:
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_train = to_categorical(y_train)

y_test = label_encoder.transform(y_test)
y_test = to_categorical(y_test)

In [ ]:
y_train_df = pd.DataFrame(y_train, columns=['class_0', 'class_1', 'class_2', 'class_3', 'class_4'])
train_df = pd.concat([X_train.reset_index(drop=True), y_train_df.reset_index(drop=True)], axis=1)
train_df.to_csv('train_data_ESI_with_robust_scaler.csv', index=False)

In [ ]:
y_test_df = pd.DataFrame(y_test, columns=['class_0', 'class_1', 'class_2', 'class_3', 'class_4'])
test_df = pd.concat([X_test.reset_index(drop=True), y_test_df.reset_index(drop=True)], axis=1)
test_df.to_csv('test_data_ESI_with_robust_scaler.csv', index=False)

In [ ]:
import joblib

joblib.dump(label_encoder, 'label_encoder_ESI.pkl')
joblib.dump(robustScaler, 'robust_scaler.pkl')